In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
import joblib

In [2]:
# Load the dataset
file_path='../data/preproccessed_coffee_prices.csv'
data = pd.read_csv(file_path, index_col='Date', parse_dates=True)

# Replace NaN and infinite values
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)  # Drop rows with NaN values
data = data[~data.index.duplicated(keep='first')]

# Display the first few rows
display(data.head())

,index,Symbol,Warehouse,Production Year,Opening Price,Closing Price,High,Low,Change,Persetntage Change,Volume (Ton),Returns,Volatility,Rolling_Corr
Date,,,,,,,,,,,,,,
2013-01-01,15576,UJMB7,BD,2004.0,795.0,795.0,795.0,795.0,5.0,0.0%,33.15,-0.135870,0.444229,0.073990
2013-01-02,16998,UHRB6,DD,2005.0,1830.0,1830.0,1830.0,1830.0,0.0,0.0%,2.55,1.909380,0.743888,0.352166
2013-01-03,12104,WLMA4,JM,2005.0,860.0,880.0,880.0,860.0,52.0,6.0%,67.98,-0.534392,0.773900,0.236028
2013-01-04,16892,UHRC4,DD,2005.0,1340.0,1340.0,1340.0,1340.0,20.0,1.0%,2.55,0.540230,0.660081,0.174173
2013-01-08,10552,WYCAQ2,DL,2005.0,1130.0,1130.0,1130.0,1130.0,20.0,1.0%,7.21,0.412500,0.432203,-0.181592


In [4]:
# Function to perform ADF test
def adf_test(series):
    # Perform the ADF test
    result = adfuller(series, autolag='AIC')
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    return result[1] > 0.05  # Returns True if the series is non-stationary

# Filter for float columns and remove those with NaN or inf values
data = data.select_dtypes(include=['float'])  # Keep only float columns
data = data.loc[:, ~data.isin([np.nan, np.inf, -np.inf]).any()]  # Drop columns with NaN or inf

print(f"Remaining columns after filtering: {data.columns.tolist()}")

# Check each series in the dataset
for column in data.columns:
    print(f'ADF Test for {column}:')
    while adf_test(data[column]):  # Perform differencing until the series is stationary
        data[column] = data[column].diff()  # Apply differencing
        data[column] = data[column].dropna()  # Remove NaN values after differencing

Remaining columns after filtering: ['Opening Price', 'Closing Price', 'High', 'Low', 'Change', 'Volume (Ton)', 'Returns', 'Volatility', 'Rolling_Corr']
ADF Test for Opening Price:
ADF Statistic: -4.639062429004748
p-value: 0.00010936782392421909
ADF Test for Closing Price:
ADF Statistic: -4.6425068859934076
p-value: 0.00010776824674179761
ADF Test for High:
ADF Statistic: -4.585036700238229
p-value: 0.0001376259003487094
ADF Test for Low:
ADF Statistic: -4.686827277494708
p-value: 8.908065084727132e-05
ADF Test for Change:
ADF Statistic: -9.514011340771143
p-value: 3.19296455325554e-16
ADF Test for Volume (Ton):
ADF Statistic: -19.186656971214553
p-value: 0.0
ADF Test for Returns:
ADF Statistic: -9.077690272603084
p-value: 4.136369509318795e-15
ADF Test for Volatility:
ADF Statistic: -3.091700001745391
p-value: 0.02717355423273736
ADF Test for Rolling_Corr:
ADF Statistic: -5.267969634639641
p-value: 6.377236991724101e-06


In [5]:
# Split data into training and test sets
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

In [6]:
correlation_matrix = train.corr()
display(correlation_matrix)

,Opening Price,Closing Price,High,Low,Change,Volume (Ton),Returns,Volatility,Rolling_Corr
Opening Price,1.000000,0.995636,0.999726,0.999559,0.090045,-0.001321,0.422820,0.014059,0.020913
Closing Price,0.995636,1.000000,0.995625,0.995656,0.030695,-0.004245,0.425783,0.013850,0.019425
High,0.999726,0.995625,1.000000,0.999280,0.090176,0.003925,0.422227,0.013617,0.019751
Low,0.999559,0.995656,0.999280,1.000000,0.089466,-0.005580,0.423649,0.014847,0.018169
Change,0.090045,0.030695,0.090176,0.089466,1.000000,-0.053394,-0.017341,-0.023337,-0.020296
Volume (Ton),-0.001321,-0.004245,0.003925,-0.005580,-0.053394,1.000000,-0.028363,0.010053,-0.008565
Returns,0.422820,0.425783,0.422227,0.423649,-0.017341,-0.028363,1.000000,0.165773,0.224789
Volatility,0.014059,0.013850,0.013617,0.014847,-0.023337,0.010053,0.165773,1.000000,0.311562
Rolling_Corr,0.020913,0.019425,0.019751,0.018169,-0.020296,-0.008565,0.224789,0.311562,1.000000


In [7]:
# Define the validate_data_quality function
def validate_data_quality(df):
    quality_checks = {
        'missing_values': df.isnull().sum().sum(),
        'duplicates': df.duplicated().sum()
    }
    return quality_checks

# Validate data quality before cleaning
quality_checks = validate_data_quality(train)
print("\nData Quality Check Results:")
print("Missing Values:", quality_checks['missing_values'])
print("Duplicate Entries:", quality_checks['duplicates'])




Data Quality Check Results:
Missing Values: 0
Duplicate Entries: 0


In [8]:
# Step 3: Check for stationarity and difference if needed
train_pca_df = pd.DataFrame(train)
for col in train_pca_df.columns:
    result = adfuller(train_pca_df[col])
    if result[1] > 0.05:
        train_pca_df[col] = train_pca_df[col].diff().dropna()  # Difference if non-stationary
display(train_pca_df.head())

,Opening Price,Closing Price,High,Low,Change,Volume (Ton),Returns,Volatility,Rolling_Corr
Date,,,,,,,,,
2013-01-01,795.0,795.0,795.0,795.0,5.0,33.15,-0.135870,0.444229,0.073990
2013-01-02,1830.0,1830.0,1830.0,1830.0,0.0,2.55,1.909380,0.743888,0.352166
2013-01-03,860.0,880.0,880.0,860.0,52.0,67.98,-0.534392,0.773900,0.236028
2013-01-04,1340.0,1340.0,1340.0,1340.0,20.0,2.55,0.540230,0.660081,0.174173
2013-01-08,1130.0,1130.0,1130.0,1130.0,20.0,7.21,0.412500,0.432203,-0.181592


In [11]:
# Save the original column names and index
column_names = train_pca_df.columns
date_index = train_pca_df.index  # Assuming train_pca_df has a datetime index

# Fit the VAR model
model = VAR(train_pca_df)
try:
    lag_order = model.select_order(maxlags=3)  # Choose maxlags based on your data size
    optimal_lag = lag_order.aic
    var_model = model.fit(optimal_lag)

    # Access the fitted values (predicted values) and assign the original column names
    fitted_values = pd.DataFrame(var_model.fittedvalues, columns=column_names)

    # Similarly, for forecasted values, you can also convert the forecast to a DataFrame
    forecast = var_model.forecast(train_pca_df.values[-optimal_lag:], steps=1000)  # 2000-step forecast

    # Create a date range for the forecasted values
    forecast_index = pd.date_range(start=date_index[-1], periods=1001, freq='D')[1:]  # Generating forecast dates

    # Create forecast DataFrame with column names and date index
    forecast_df = pd.DataFrame(forecast, columns=column_names, index=forecast_index)

    # Save the forecast data to CSV
    forecast_df.to_csv('../data/model_forecast.csv', index=True)

    display("Forecasted Values:\n", forecast_df)

except np.linalg.LinAlgError:
    print("Handling LinAlgError by refitting with a lower lag or regularization.")

c:\Users\Administrator\miniconda3\envs\jojo\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


'Forecasted Values:\n'

,Opening Price,Closing Price,High,Low,Change,Volume (Ton),Returns,Volatility,Rolling_Corr
2018-05-09,1129.182624,1130.255421,1131.907378,1122.340778,29.708249,26.672385,0.215082,0.435019,0.103530
2018-05-10,1123.142185,1125.278778,1129.971436,1121.289016,32.787691,29.895362,0.164539,0.522349,0.144727
2018-05-11,1091.124964,1088.350962,1091.468525,1078.966803,38.407663,24.115654,0.202041,0.602499,0.146172
2018-05-12,1063.711346,1061.838986,1066.477756,1059.492076,27.861441,24.814835,0.105544,0.581367,0.131433
2018-05-13,1072.784918,1071.681886,1075.665923,1069.822281,28.816512,26.340703,0.111783,0.605410,0.140850
...,...,...,...,...,...,...,...,...,...
2021-01-28,1017.175766,1015.956028,1019.308938,1015.064215,24.877217,23.880379,0.134032,0.711932,0.147654
2021-01-29,1017.175766,1015.956028,1019.308938,1015.064215,24.877217,23.880379,0.134032,0.711932,0.147654
2021-01-30,1017.175766,1015.956028,1019.308938,1015.064215,24.877217,23.880379,0.134032,0.711932,0.147654
2021-01-31,1017.175766,1015.956028,1019.308938,1015.064215,24.877217,23.880379,0.134032,0.711932,0.147654
